- Fine-tune multi-lingual - with both Hindi and Nepali
- Fine tune with Hindi run inference on Nepali 
- Fine tune with Nepali run inference on Hindi
- Fine tune with Nepali run inference on Nepali
- Fine tune with Hindi run inference on Hindi

In [1]:
from kaggle_secrets import UserSecretsClient


In [2]:
user_secrets = UserSecretsClient()
token = user_secrets.get_secret("GITHUB_KEY")

In [3]:
!git clone https://{token}@github.com/shreeya-dhakal/llama-3-finetune.git

Cloning into 'llama-3-finetune'...
remote: Enumerating objects: 132, done.
remote: Counting objects: 100% (132/132), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 132 (delta 65), reused 74 (delta 19), pack-reused 0
Receiving objects: 100% (132/132), 27.96 MiB | 18.88 MiB/s, done.
Resolving deltas: 100% (65/65), done.


In [4]:
%%capture
!mamba install --force-reinstall aiohttp -y
!pip install -U "xformers<0.0.26" --index-url https://download.pytorch.org/whl/cu121
!pip install "unsloth[kaggle-new] @ git+https://github.com/unslothai/unsloth.git"

# Temporary fix for https://github.com/huggingface/datasets/issues/6753
!pip install datasets==2.16.0 fsspec==2023.10.0 gcsfs==2023.10.0

In [5]:
import os
import wandb

os.environ["WANDB_PROJECT"]="llama-finetune-nepali"
os.environ["WANDB_LOG_MODEL"] = "checkpoint"
wandb_token = user_secrets.get_secret("WANDB_KEY")
wandb.login(key= wandb_token)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [7]:
from datasets import load_dataset, load_metric, concatenate_datasets
import sys
import os
sys.path.append('/kaggle/working/llama-3-finetune/src/')
from data.data_sampler import sample_data
from unsloth import FastLanguageModel
import torch
import re

In [9]:
split_ratio = 0.2
seed = 42

In [10]:
dataset_name = "CohereForAI/aya_dataset"
aya_dataset = load_dataset(dataset_name)
dataset_type = "aya"

Generating train split:   0%|          | 0/202362 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1750 [00:00<?, ? examples/s]

In [11]:
lang_multi = "Nepali"
aya_nepali_train, _ = sample_data(dataset_name, dataset_type, split_ratio, seed, output_dir=None, lang_multi=lang_multi)
aya_nepali_train

Filter:   0%|          | 0/202362 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1750 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4002 [00:00<?, ? examples/s]

Dataset({
    features: ['inputs', 'targets', 'language', 'language_code', 'annotation_type', 'user_id'],
    num_rows: 3201
})

In [12]:
lang_multi = "Hindi"
aya_hindi_train, _ = sample_data(dataset_name, dataset_type, split_ratio, seed, output_dir=None, lang_multi=lang_multi)
aya_hindi_train

Filter:   0%|          | 0/202362 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1750 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1153 [00:00<?, ? examples/s]

Dataset({
    features: ['inputs', 'targets', 'language', 'language_code', 'annotation_type', 'user_id'],
    num_rows: 922
})

In [13]:
aya_nepali_train = aya_nepali_train.rename_columns(
            {"inputs": "input", "targets": "output"})
new_column_data = [""] * len(aya_nepali_train)  # Create a list with the same length as the dataset
aya_nepali_train = aya_nepali_train.add_column("instruction", new_column_data)
aya_nepali_train = aya_nepali_train.remove_columns(['language', 'language_code', 'annotation_type', 'user_id'])
aya_nepali_train

Flattening the indices:   0%|          | 0/3201 [00:00<?, ? examples/s]

Dataset({
    features: ['input', 'output', 'instruction'],
    num_rows: 3201
})

In [14]:
aya_hindi_train = aya_hindi_train.rename_columns(
            {"inputs": "input", "targets": "output"})
new_column_data = [""] * len(aya_hindi_train)
aya_hindi_train = aya_hindi_train.add_column("instruction", new_column_data)
aya_hindi_train = aya_hindi_train.remove_columns(['language', 'language_code', 'annotation_type', 'user_id'])
aya_hindi_train

Flattening the indices:   0%|          | 0/922 [00:00<?, ? examples/s]

Dataset({
    features: ['input', 'output', 'instruction'],
    num_rows: 922
})

In [15]:
dataset_name = "Saugatkafley/alpaca-nepali-sft"

In [16]:
alpaca_nepali_train, _ = sample_data(dataset_name, "alpaca", split_ratio, seed, output_dir=None, lang_multi=None)
alpaca_nepali_train

Generating train split:   0%|          | 0/52005 [00:00<?, ? examples/s]

Filter:   0%|          | 0/52005 [00:00<?, ? examples/s]

Filter:   0%|          | 0/51963 [00:00<?, ? examples/s]

Filter:   0%|          | 0/51963 [00:00<?, ? examples/s]

Dataset({
    features: ['instruction', 'input', 'output', 'id'],
    num_rows: 41567
})

In [17]:
dataset_name = "iamshnoo/alpaca-cleaned-hindi"

In [18]:
alpaca_hindi_train, _ = sample_data(dataset_name, "alpaca", split_ratio, seed, output_dir=None, lang_multi=None)
alpaca_hindi_train

Generating train split:   0%|          | 0/51760 [00:00<?, ? examples/s]

Filter:   0%|          | 0/51760 [00:00<?, ? examples/s]

Filter:   0%|          | 0/51760 [00:00<?, ? examples/s]

Filter:   0%|          | 0/51760 [00:00<?, ? examples/s]

Dataset({
    features: ['input', 'instruction', 'output'],
    num_rows: 41408
})

In [19]:
nepali_train = concatenate_datasets([aya_nepali_train, alpaca_nepali_train])
nepali_train = nepali_train.shuffle(seed=42)
nepali_train

Dataset({
    features: ['input', 'output', 'instruction', 'id'],
    num_rows: 44768
})

In [20]:
hindi_train = concatenate_datasets([aya_hindi_train, alpaca_hindi_train])
hindi_train = hindi_train.shuffle(seed=42)
hindi_train

Dataset({
    features: ['input', 'output', 'instruction'],
    num_rows: 42330
})

In [19]:
# both_train = concatenate_datasets([nepali_train, hindi_train])
# both_train = both_train.shuffle(seed=42)
# both_train

In [21]:
#Nepali
prompt = """Below is an instruction in Nepali that describes a task, paired with an input also in Nepali that provides further context. Write a response in Nepali that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [22]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

tokenizer.add_special_tokens({"pad_token": "<|reserved_special_token_0|>"})
model.config.pad_token_id = tokenizer.pad_token_id
tokenizer.padding_side = 'right'

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.7
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.2+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.25.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

In [23]:
print(f"Pad Token id: {tokenizer.pad_token_id} and Pad Token: {tokenizer.pad_token}")
print(f"EOS Token id: {tokenizer.eos_token_id} and EOS Token: {tokenizer.eos_token}")

Pad Token id: 128002 and Pad Token: <|reserved_special_token_0|>
EOS Token id: 128001 and EOS Token: <|end_of_text|>


In [24]:
#LoRA
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [25]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN


def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

train_data = nepali_train
train_data = train_data.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/44768 [00:00<?, ? examples/s]

In [28]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_data,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 100,
        save_steps=300,                 ### Checkpoint will be save after every 500 steps
        optim = "adamw_8bit",
        max_steps = 10392,
        weight_decay = 0.01,
        report_to="wandb", 
        run_name="llama3_alpaca_nep", 
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs_alpaca_nep",   # Saving the checkpoints to outputs folder
    ),
)

Map (num_proc=2):   0%|          | 0/44768 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [29]:
trainer_stats = trainer.train()
#train_data = 11192

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 44,768 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 10,392
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss


KeyboardInterrupt: 

In [ ]:
wandb.finish()